### Hugging Face Pipeline Integration
The notebook shows how to run Hugging Face pipelines with Aana SDK. The notebook uses image captioning pipeline to showcase the integration.

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Create Aana SDK and connect to the cluster.

In [2]:
from aana.sdk import AanaSDK


aana_app = AanaSDK().connect()

/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-24 14:13:36,633	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(
2024-06-24 14:13:40,438	WARNING services.py:2009 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free u

Deploy Hugging Face pipeline as Aana deployment.

In [3]:
from aana.deployments.hf_pipeline_deployment import (
    HfPipelineConfig,
    HfPipelineDeployment,
)

hf_pipeline_deployment = HfPipelineDeployment.options(
    num_replicas=1,                             # The number of replicas of the model to deploy
    ray_actor_options={"num_gpus": 0.25},       # Allocate 0.25 GPU, should be > 0 if the model requires GPU
    user_config=HfPipelineConfig(
        model_id="ydshieh/vit-gpt2-coco-en",    # The model ID from the Hugging Face model hub
        model_kwargs={},                        # Extra model parameters
    ).model_dump(mode="json"),
)

aana_app.register_deployment(
    name="image_captioning_model",              # Name of the deployment, which will be using to access the deployment
    instance=hf_pipeline_deployment,            # Instance of the deployment that we just created above
    deploy=True                                 # Tell Aana to deploy the component immediately instead of waiting aana_app.deploy()
)

/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_kwargs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
The new client HTTP config differs from the existing one in the following fields: ['location']. The new HTTP config is ignored.
2024-06-24 14:13:49,684	INFO handle.py:126 -- Created DeploymentHandle 'gd8uy5xi' for Deployment(name='HfPipelineDeployment', app='image_captioning_model').
2024-06-24 14:13:49,686	INFO handle.py:126 -- Created DeploymentHandle 'jegi4gws' for Dep

Create AanaDeploymentHandle to connect to the deployment. We use the same name `image_captioning_model` that we used while deploying the model.

In [4]:
from aana.deployments.aana_deployment_handle import AanaDeploymentHandle

handle = await AanaDeploymentHandle.create("image_captioning_model")

2024-06-24 14:14:04,838	INFO handle.py:126 -- Created DeploymentHandle 'xoc62018' for Deployment(name='HfPipelineDeployment', app='image_captioning_model').


2024-06-24 14:14:04,854	INFO pow_2_scheduler.py:260 -- Got updated replicas for Deployment(name='HfPipelineDeployment', app='image_captioning_model'): {'3l9u324s'}.


Load an image from the URL.

In [5]:
import requests
from PIL import Image as PilImage

url = "http://images.cocodataset.org/val2017/000000039769.jpg"

image = PilImage.open(requests.get(url, stream=True).raw)

And call the model remotely with the handle we just created.

In [6]:
await handle.call(images=[image], generate_kwargs={"max_new_tokens": 200})

2024-06-24 14:14:05,617	INFO handle.py:126 -- Created DeploymentHandle '5d8fay22' for Deployment(name='HfPipelineDeployment', app='image_captioning_model').


[[{'generated_text': 'a cat laying on a blanket next to a cat laying on a bed '}]]

In [7]:
await handle.call(image, generate_kwargs={"max_new_tokens": 200})

2024-06-24 14:14:06,282	INFO handle.py:126 -- Created DeploymentHandle 'ip14x66z' for Deployment(name='HfPipelineDeployment', app='image_captioning_model').


[{'generated_text': 'a cat laying on a blanket next to a cat laying on a bed '}]

In [8]:
await handle.call(url, generate_kwargs={"max_new_tokens": 200})

2024-06-24 14:14:06,663	INFO handle.py:126 -- Created DeploymentHandle 'rmrlx82x' for Deployment(name='HfPipelineDeployment', app='image_captioning_model').


[{'generated_text': 'a cat laying on a blanket next to a cat laying on a bed '}]

In [9]:
urls = [
    "https://huggingface.co/datasets/sayakpaul/sample-datasets/resolve/main/pokemon.png",
    "http://images.cocodataset.org/val2017/000000039769.jpg",
]

In [10]:
await handle.call(urls, generate_kwargs={"max_new_tokens": 200})

2024-06-24 14:14:07,714	INFO handle.py:126 -- Created DeploymentHandle 'mh78lhia' for Deployment(name='HfPipelineDeployment', app='image_captioning_model').


[[{'generated_text': 'a stuffed animal with a heart on it '}],
 [{'generated_text': 'a cat laying on a blanket next to a cat laying on a bed '}]]

Now you can create an Aana Endpoint to expose the deployment as an API endpoint.